In [3]:
import re
import spacy
import pandas as pd

#from PIL import Image
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
import string
import itertools
import en_core_web_lg
import copy

from presidio_analyzer import (
    Pattern,
    PatternRecognizer,
    RecognizerRegistry,
    AnalyzerEngine
)

import warnings
warnings.filterwarnings('ignore')

In [4]:
# load SpaCy English language model
nlp = en_core_web_lg.load()

# Identification PII entities

# Patients

In [5]:
# String to test "PERSON" and "PATIENT PERSON" entities
text_patient2ident="Patient: John, Doe MSTR UR No.: 1711111 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Evergreen Medical Center. PatientName: Anna-Marie, O'Connor MISS UR No.: 1711112 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Silvercrest Regional Hospital. Patient: Michael, MSTR UR No.: 1711113 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Starlight Health Institute. Patient: Jane-Doe, Christoper MSTR UR No.: 1711114 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Maplewood General Hospital. Patient: Laura, O'Hara MISS UR No.: 1711115 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Cedarbrook Medical Complex. Patient: Anna, baby of O'Hara MISS UR No.: 1711116 Admit: 01/01/2020 DOB: 31/12/1999 Disch: 06/01/2020 Facility: Bright Horizons Medical Center."

### Default identifier

In [6]:
# Create an instance of the AnalyzerEngine
# The AnalyzerEngine is used to analyze text, recognizing and extracting entities based on predefined patterns and rules.
analyzer = AnalyzerEngine()
# Perform the analysis on the text, focusing on extracting "PERSON" entities
results_pat_ident = analyzer.analyze(text=text_patient2ident,
                               entities=["PERSON"],
                               language='en') 

In [7]:
results_pat_ident

[type: PERSON, start: 9, end: 13, score: 0.85,
 type: PERSON, start: 141, end: 151, score: 0.85,
 type: PERSON, start: 285, end: 292, score: 0.85,
 type: PERSON, start: 414, end: 422, score: 0.85,
 type: PERSON, start: 424, end: 445, score: 0.85,
 type: PERSON, start: 555, end: 560, score: 0.85,
 type: PERSON, start: 689, end: 693, score: 0.85,
 type: PERSON, start: 703, end: 717, score: 0.85]

### Customized identifier

In [8]:
# List of context words that indicate patint's names
context_words_patient=['Patient Name:','PatientName:','Patient' ]

In [9]:
# Define the regex pattern for matching patient names with a suffix (e.g., "MSTR", "MISS")
regex_1 = r"([A-Z][a-zA-Z][-'']?[A-Z]?[a-zA-Z]*)+,\s*([A-Za-z-' ]+)(\s)(MSTR|MISS)" 
# Define the regex pattern for matching patient names with the "Patient" keyword as a prefix
regex_2 = r"(Patient)(:)?\s([A-Z][a-zA-Z][-'']?[A-Z]?[a-zA-Z]*)+,\s*([A-Za-z-' ]+)"

# Create a pattern object for matching patient names 
patient_name_pattern_postfix = Pattern(name="patient name (weak)", regex=regex_1, score=0.90)
patient_name_pattern_prefix = Pattern(name="patient name (weak)", regex=regex_2, score=0.90)

# Define the recognizer with the defined patterns and the context words for patients
patient_name_recognizer_w_context = PatternRecognizer(
    supported_entity="PATIEN PERSON",
    patterns=[patient_name_pattern_postfix,patient_name_pattern_prefix],
    context=context_words_patient,
    
)

# Create a recognizer registry to store multiple recognizers
analyzer = AnalyzerEngine()
# Add the recognizer to the registry
registry = RecognizerRegistry()
registry.add_recognizer(patient_name_recognizer_w_context)
# Create an analyzer engine that uses the registry to analyze text
analyzer = AnalyzerEngine(registry=registry)

In [10]:
# test recognizer
results = analyzer.analyze(text=text_patient2ident,  entities=["PATIEN PERSON"],   language='en')  
results

[type: PATIEN PERSON, start: 0, end: 29, score: 0.9,
 type: PATIEN PERSON, start: 141, end: 166, score: 0.9,
 type: PATIEN PERSON, start: 276, end: 304, score: 0.9,
 type: PATIEN PERSON, start: 405, end: 445, score: 0.9,
 type: PATIEN PERSON, start: 546, end: 579, score: 0.9,
 type: PATIEN PERSON, start: 680, end: 720, score: 0.9]

## Clinicians (customized identifier)

In [11]:
# List of context words that indicate the clinicians
contect_words_clinicians=['Attending Physician:','Updated By:','Result Comment:','Order:','Perform:','Witness:','VERIFY:','Admin Details: (Auth)','Admin Person:','Perform Information:','Sign Information:','Examination Verified By:','Notes written by','[Physician name] Neonatal Registrar','Auth (Verified)','RWR ([Physician name]/[Physician name])','PNW Review - ','[Physician name] PNW Registrar','General Practitioner    Name:','Author   Name:','By:','Newborn Examination Verified By:','[Physician name] - DIET','[Physician name] - INTMO', 'Examination Verified By', 'Treating Clinician']

In [12]:
# Regular expression patterns for various clinician-related names and titles

#regex_cl_1 - this regex pattern matches clinician names followed by their titles or roles e.g."Jane, Doe Registered Nurse"
regex_cl_1 = r"([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)+,\s*([A-Za-z-']+)(\s-*\s*)(SP|HMO|RM|Registered Midwife|RN|Registered Nurse|SMO|Senior Medical Officer|INTMO|International Medical Officer|REGMO|Registrar Medical Officer|GP|General Practitioner|Paediatrician|Pediatrician|OB/GYN|OB|GYN|Obstetrician/Gynecologist|ENT|Ear, Nose, and Throat Specialist|Oncologist|Oncologist|Surgeon|Surgeon|Intern|Medical Intern|Resident|Medical Resident|Fellow|Medical Fellow|OT|Occupational Therapist|PT|Physical Therapist|SLP|Speech-Language Pathologist|RT|Respiratory Therapist|DIET|Dietitian|MT|Medical Technologist|ST|Surgical Technologist|NMT|Nuclear Medicine Technologist|MRT|Medical Radiation Technologist|CNA|Certified Nursing Assistant|EMT|Emergency Medical Technician|Paramedic|Paramedic|LPN|Licensed Practical Nurse|CNA|Certified Nursing Assistant|EN|Enrolled Nurse|EEN|Endorsed Enrolled Nurse|PHARM|PharmD (Doctor of Pharmacy)|RPh|Registered Pharmacist|FNP|Family Nurse Practitioner|CRNA|Certified Registered Nurse Anesthetist|Audiologist|Audiologist|Podiatrist|Podiatrist|Social Worker|Social Worker|Clinical Psychologist|Clinical Psychologist|Counsellor|Counsellor|Support and Administrative Roles|Health Information Manager|Health Information Manager|Medical Receptionist|Medical Receptionist|Medical Administrator|Medical Administrator)"
# regex_cl_2 - this pattern matches phrases like CWR (Johnson/Brown)
regex_cl_2 = r"((RWR|CWR)(\s)?\(([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)(?:(/|,)(\s)?[A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*){1,5}\))"
#regex_cl_3 - his regex is designed to match phrases in a format like Dr. John Smith
regex_cl_3 = r"(Verified By|Written By|written by|Treating Clinician|Dr|Dr.|Review with|RNW Review-)(:)?\s([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)+(?:\s[a-zA-Z]+)?(,)? [a-zA-Z]+(?:\s[a-zA-Z]+)?"
# regex_cl_4 - matches clinician names followed by an optional number, usually identifying a case or record e.g. Sara Green, Black #11223344
regex_cl_4 = r"([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)+(?:\s[a-zA-Z]+)?(,)? [a-zA-Z]+(?:\s[a-zA-Z]+)? #\d{3,10}"
#regex_cl_5 - matches patterns where "Review with" is followed by one or more clinician names and their roles
regex_cl_5 = r"(Review with)(\s)?([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)(?:(/|,)(\s)?[A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*){1,5}"
# This regex is used to match "Action Personnel" followed by the name of the assigned clinician e.g.Action Personnel John Doe
regex_cl_6 = r"(Action Personnel)(\s)?([A-Z][a-zA-Z]*[-'']?[A-Z]?[a-zA-Z]*)"
#Matches clinician names and their roles in the format "Name - Role"
regex_cl_7 = r"([A-Za-z-']+)(\s-\s*)(SP|HMO|RM|Registered Midwife|RN|Registered Nurse|SMO|Senior Medical Officer|INTMO|International Medical Officer|REGMO|Registrar Medical Officer|GP|General Practitioner|Paediatrician|Pediatrician|OB/GYN|OB|GYN|Obstetrician/Gynecologist|ENT|Ear, Nose, and Throat Specialist|Oncologist|Oncologist|Surgeon|Surgeon|Intern|Medical Intern|Resident|Medical Resident|Fellow|Medical Fellow|OT|Occupational Therapist|PT|Physical Therapist|SLP|Speech-Language Pathologist|RT|Respiratory Therapist|DIET|Dietitian|MT|Medical Technologist|ST|Surgical Technologist|NMT|Nuclear Medicine Technologist|MRT|Medical Radiation Technologist|CNA|Certified Nursing Assistant|EMT|Emergency Medical Technician|Paramedic|Paramedic|LPN|Licensed Practical Nurse|CNA|Certified Nursing Assistant|EN|Enrolled Nurse|EEN|Endorsed Enrolled Nurse|PHARM|PharmD (Doctor of Pharmacy)|RPh|Registered Pharmacist|FNP|Family Nurse Practitioner|CRNA|Certified Registered Nurse Anesthetist|Audiologist|Audiologist|Podiatrist|Podiatrist|Social Worker|Social Worker|Clinical Psychologist|Clinical Psychologist|Counsellor|Counsellor|Support and Administrative Roles|Health Information Manager|Health Information Manager|Medical Receptionist|Medical Receptionist|Medical Administrator|Medical Administrator)"

# Create a pattern objects for matching patient names
clinicians_names_pattern_1 = Pattern(name="clinician name with position (weak)", regex=regex_cl_1, score=0.8)
clinicians_names_pattern_2 = Pattern(name="clinician name set of last names (weak)", regex=regex_cl_2, score=0.85)
clinicians_names_pattern_3 = Pattern(name="clinician name without position(weak)", regex=regex_cl_3, score=0.7)
clinicians_names_pattern_4 = Pattern(name="clinician name with number", regex=regex_cl_4, score=0.8)
clinicians_names_pattern_5 = Pattern(name="clinician name with number", regex=regex_cl_5, score=0.8)
clinicians_names_pattern_6 = Pattern(name="clinician name with number", regex=regex_cl_6, score=0.75)
clinicians_names_pattern_7 = Pattern(name="clinician name with number", regex=regex_cl_7, score=0.9)

# Define the recognizer with the defined pattern and context words
clinicians_name_recognizer_w_context = PatternRecognizer(
    supported_entity="CLINICIAN PERSON",
    patterns=[clinicians_names_pattern_1,clinicians_names_pattern_2,clinicians_names_pattern_3,clinicians_names_pattern_4,clinicians_names_pattern_5,clinicians_names_pattern_6,clinicians_names_pattern_7], #,clinicians_names_pattern_3,clinicians_names_pattern_1,clinicians_names_pattern_2,] clinicians_names_pattern_1, clinicians_names_pattern_2,
    context=contect_words_clinicians,
    global_regex_flags = 1,
    
    
)

In [13]:
# test recognizer
text_clinit2ident= "RWR (O'Neill, Green). CWR (Johnson/Brown). Review with: Dr. Robert White. Written By: Alice Smith. Action Personnel John Doe. Sara Green, Black #11223344. Alice Smith - Registered Nurse. Dr. John Doe - GP"

test_result=clinicians_name_recognizer_w_context.analyze(text=text_clinit2ident,
                                           entities=["CLINICIAN PERSON"],)
test_result

[type: CLINICIAN PERSON, start: 161, end: 185, score: 0.9,
 type: CLINICIAN PERSON, start: 196, end: 204, score: 0.9,
 type: CLINICIAN PERSON, start: 0, end: 20, score: 0.85,
 type: CLINICIAN PERSON, start: 22, end: 41, score: 0.85,
 type: CLINICIAN PERSON, start: 126, end: 153, score: 0.8,
 type: CLINICIAN PERSON, start: 99, end: 120, score: 0.75,
 type: CLINICIAN PERSON, start: 56, end: 72, score: 0.7,
 type: CLINICIAN PERSON, start: 74, end: 97, score: 0.7,
 type: CLINICIAN PERSON, start: 187, end: 199, score: 0.7]

# Address VIC (customized identifier)

In [14]:
#list of VIC suburbs
australia_state_suburb_vic=['FREEBURGH','MAUDE','DUNLUCE','CLYDE','BRIGHTON','PASCOE VALE SOUTH','SALE','GOLDEN BEACH','CAPE CLEAR','NILMA NORTH','GLEN HUNTLY','ACHERON','AMPHITHEATRE','NOORINBEE','SUMMERLANDS','WOODVALE','ALLAMBEE','HADFIELD','BUNGEET WEST','STRATHMORE','LAKE TYERS BEACH','KANUMBRA','DIAMOND CREEK','DALMORE','PORCUPINE RIDGE','MERRIANG','CHARAM','HEATHMONT','DUNDONNELL','FORBES','WEERING','GLENBURN','BUNGADOR','BATHUMI','GNOTUK','WHITFIELD','MINHAMITE','DRUMMARTIN','FLAGSTAFF','LAKE WELLINGTON','GLENLYON','FRESHWATER CREEK','HOMEWOOD','ROCKLANDS','CRANBOURNE SOUTH','STOCKYARD HILL','TATONG','MURRINDINDI','FOOTSCRAY','GLENLOFTY','MIGA LAKE','NARING','RUTHERGLEN','MENZIES CREEK','BRIAGOLONG','FINGAL','RICHMOND PLAINS','RYTHDALE','TARWIN','MOLESWORTH','GRENVILLE','YERING','DUTTON WAY','IRYMPLE','MURNUNGIN','THORNTON','ARCHDALE','FLYNN','ENGLEFIELD','SUNDERLAND BAY','FOSTER NORTH','ONDIT','ILLOWA','DUMOSA','BEALIBA','GLENGARRY NORTH','GLENPATRICK','REDBANK','QUAMBATOOK','BURKES BRIDGE','MUSK VALE','BEACONSFIELD UPPER','GOWANBRAE','WINNAP','WATTLE GLEN','ORRVALE','CURYO','BYLANDS','ALLANS FLAT','CHEPSTOWE','YANAKIE','COBRAM EAST','GARIBALDI','CORINELLA','RYANSTON','MIDDLE TARWIN','TEAL POINT','GOROKE','CAMPBELLFIELD','NARRE WARREN SOUTH','BAEL BAEL','RUPANYUP','BROADWATER','TOOMBON','EDI','SARGOOD','BOINKA','BORONIA','UNDERA','BALACLAVA','SEA LAKE','CLARETOWN','TEDDYWADDY','BRAESIDE','FITZROY NORTH','GATEWAY ISLAND','ALBACUTYA','GATUM','HALLSTON','KIEWA','WENDOUREE','AGNES','ST ARNAUD NORTH','KULWIN','LAEN','BUNYIP','MENTONE','BYAWATHA','FERNDALE','CALROSSIE','MARKWOOD','BLACK ROCK','EASTVILLE','ANAKIE','WALHALLA','SYLVATERRE','RANCEBY','PIEDMONT','NORTHCOTE','IONA','GRAHAMVALE','WARNCOORT','MURCHISON','WEATHERBOARD','AIRLY','BENLOCH','SCARSDALE','GLEN VALLEY','RESEARCH','BEEAC','EUROBIN','BOOLE POOLE','KURRACA WEST','DELACOMBE','YAWONG HILLS','TELFORD','STRATHFIELDSAYE','HMAS CERBERUS','BUMBERRAH','ST ARNAUD EAST','EVERSLEY','MINMINDIE','TAYLORS HILL','WANDELLA','POWERS CREEK','WAYGARA','MYAMYN','YARRAWONGA SOUTH','SMEATON','NEERIM SOUTH','DEVON NORTH','COSGROVE SOUTH','TOTTENHAM','BASS','TOOLAMBA','SAWMILL SETTLEMENT','PARADISE','DARLINGTON','SPRINGBANK','FRANKSTON','BLACKBURN SOUTH','NURRABIEL','CARDROSS','HILL END','NYAH','TAYLORS LAKES','TYNONG NORTH','MARTHAVALE','YARRABERB','CROSSOVER','BARUNAH PARK','KALKEE','HARSTON','CARLSRUHE','GNARWARRE','KALORAMA','BRIMBOAL','MULWALA','CARGERIE','COWWARR','MORRISONS','LITCHFIELD','THORNHILL PARK','TOOLONG','WONGARRA','DRUMMOND','TOONGABBIE','SANDON','NEWLYN','YANDOIT HILLS','LABERTOUCHE','STONEHAVEN','KOOROOL','JINDIVICK','VERVALE','SHAYS FLAT','HAVILAH','DURHAM LEAD','NETHERBY','CHRISTMAS HILLS','WRIXON','SPRINGDALLAH','POOWONG NORTH','LLANELLY','MOOROOLBARK','TUBBUT','LANGSBOROUGH','RIACHELLA','LAKE EPPALOCK','YACKANDANDAH','CARRON','KAWARREN','APPIN SOUTH','DINGLEY VILLAGE','ROSS CREEK','MEREDITH','DOVETON','ARCADIA','BINDI','MORTLAKE','CHUM CREEK','CARNGHAM','MACS COVE','PURDEET','YAMBUK','CORINDHAP','CLAYTON SOUTH','BREAKWATER','HORDERN VALE','INVERGORDON','SANDFORD','DHURRINGILE','ROCHFORD','WICKLIFFE','TURRIFF','TORRUMBARRY','GREENSBOROUGH','LINGA','COBURG NORTH','PIPERS CREEK','MYSIA','MOUNT BRUNO','NURCOUNG','WILLANGIE','HALLORA','ARAWATA','IVANHOE','CARRANBALLAC','BORNES HILL','KINGSBURY','ST KILDA WEST','WATTLE BANK','CLIFTON SPRINGS','OAKLEIGH SOUTH','BINGINWARRI','OZENKADNOOK','SEAVIEW','CONCONGELLA','BASALT','DURHAM OX','MYRTLEFORD','BENTLEIGH EAST','GARFIELD NORTH','NEWHAVEN','MUSKERRY','TAHARA WEST','LILLICO','CAULFIELD SOUTH','TATYOON','WINTON NORTH','LINDSAY POINT','LYNBROOK','LUCAS','HAZELWOOD SOUTH','CARRAJUNG SOUTH','STRATFORD','PIONEER BAY','KENMARE','LONGWARRY NORTH','WINLATON','WHITTINGTON','TANWOOD','WAUBRA','LYSTERFIELD','GISBORNE','MOUNT BURNETT','UPPER RYANS CREEK','HIGHTON','WARRANDYTE','OUYEN','CAPELS CROSSING','PATTERSON LAKES','WILDWOOD','IRISHTOWN','LAVERTON','THOONA','DROPMORE','BRUNSWICK EAST','HARMERS HAVEN','CLARENDON','AMHERST','BOTANIC RIDGE','PARKDALE','REEDY LAKE','GHERANG','FRAMLINGHAM','CATUMNAL','MOUNT BUTE','HALLAM','MONT ALBERT','WEERITE','TAHARA BRIDGE','NARRE WARREN NORTH','PITTONG','SOUTH MELBOURNE','POINT WILSON','PHEASANT CREEK','MERBEIN WEST','HAPPY VALLEY','TRUGANINA','SHEANS CREEK','KYALITE','FITZROY','LEOPOLD','HESKET','TURTONS CREEK','BONSHAW','MELWOOD','WHEATSHEAF','SEBASTOPOL','JACK RIVER','COBUNGRA','CORNISHTOWN','YALCA','COLLINGWOOD','TUERONG','MALVERN','BOORHAMAN EAST','BANNERTON','FAWCETT','TERIP TERIP','LANG LANG EAST','ALVIE','OXLEY FLATS','PORT WELSHPOOL','COOMOORA','ALBION','WAITCHIE','MARAMINGO CREEK','MEERING WEST','ALPHINGTON','MILLOO','BAGSHOT','DARNUM','JACOB CREEK','KYNETON SOUTH','QUANDONG','WOORARRA EAST','GOULBURN WEIR','GREDGWIN','WYCHEPROOF SOUTH','LEARMONTH','CALLAWADDA','NARRAPORT','ARNOLD','STRANGWAYS','NORADJUHA','BAXTER','SOUTH DUDLEY','COLAC EAST','KNOXFIELD','CARRUM','KINGSVILLE','HESSE','DINGWALL','ELINGAMITE NORTH','WULGULMERANG','LEONGATHA NORTH','GEELONG','CROSSLEY','MOUNT MACEDON','NIRRANDA','DERRINALLUM','SUNSHINE','MILLTOWN','WOOLENOOK','BEAUMARIS','BULN BULN EAST','GLENALADALE','TOWANINNY','HOWQUA','SOUTHBANK','LEICHARDT','TRAFALGAR','TARILTA','TOWONG','MOE','CALULU','TARCOMBE','VASEY','DARTMOOR','MUCKLEFORD SOUTH','YARRA JUNCTION','BAMAWM EXTENSION','YARROWEYAH','HUNTLY NORTH','MILLBROOK','BROOKFIELD','WOORARRA WEST','JERICHO','TUTYE','SCORESBY','BANNOCKBURN','BOUNDARY BEND','ROBINVALE','EMU CREEK','RICH AVON','KONONGWOOTONG','MURRAYDALE','RAINBOW','TUROAR','BAIRNSDALE','BOHO','CALLIGNEE SOUTH','WAHRING','ST KILDA EAST','COBRICO','BARRAPORT WEST','ELIZABETH ISLAND','CHESHUNT','BUCKLAND','CAPEL SOUND','LONGWARRY','DEEP LEAD','PINE MOUNTAIN','GLENAIRE','GLENDARUEL','WINDSOR','MORDIALLOC','LANCEFIELD','WALLACEDALE','SWIFTS CREEK','NERRENA','BOSTOCKS CREEK','BROUGHTON','OUTTRIM','TABLE TOP','JOYCES CREEK','WHARPARILLA','MOUNT COLE','WENTWORTH','TAMLEUGH','BALLARAT EAST','KARIAH','JACANA','WOORINEN NORTH','GHERINGHAP','CANNIE','BARNADOWN','HARRIETVILLE','TOL TOL','NERRIN NERRIN','PICOLA','KIRKSTALL','CONNEWARRE','MANNERIM','BUXTON','ESSENDON FIELDS','KOOROOMAN','MOORNGAG','YEUNGROON','YARRAM','WYELANGTA','CABARITA','KERANG EAST','NEWBRIDGE','MASSEY','TOORA NORTH','SANDY POINT','CHILLINGOLLAH','SAMARIA','BRAYBROOK','HURSTBRIDGE','INGLEWOOD','FRANKLINFORD','BINGO MUNJIE','RUBICON','KORUMBURRA','WONWONDAH','EDGECOMBE','BOLWARRA','BUNDALONG SOUTH','GEORGES CREEK','COTSWOLD','DOOEN','NUNGATTA','EPPALOCK','CRYMELON','TEMPY','CHURCHILL ISLAND','EAGLEHAWK','HIAMDALE','BIMBOURIE','AVONMORE','MAINTONGOON','ALMURTA','HAZEL PARK','NEW GISBORNE','ALBERTON','GILLUM','MERTON','MOUNT CAMEL','KANYAPELLA','PIGEON PONDS','SANDRINGHAM','COORIEMUNGLE','LYONS','WHITTLESEA','MYOLA','DONNYBROOK','MILL PARK','CASHMORE','TRENTHAM EAST','STUART MILL','MYSTIC PARK','MOUNT ELIZA','LEITCHVILLE','GOSCHEN','ST LEONARDS','ARCHIES CREEK','ECHUCA','LOCKINGTON','CODRINGTON','MARLBED','BOOLARRA SOUTH','HOMERTON','UPPER GUNDOWRING','GERANGAMETE','CHILTERN VALLEY','APPIN','EMERALD','KOOREH','TATURA EAST','EAST BAIRNSDALE','POLISBET','CLOVERLEA','LAEN NORTH','TALBOT','DROMANA','MOORA','TALLANGATTA EAST','WOODSTOCK WEST','KOONDA','CALDERMEADE','YELLINGBO','HOWQUA HILLS','HEATHCOTE SOUTH','JEFFCOTT NORTH','YANNATHAN','GERAHMIN','INDENTED HEAD','DIXIE','CARRAJUNG','HEDLEY','ALBERT PARK','MOORALLA','LAANG','CRANBOURNE NORTH','INVERMAY PARK','WALDARA','ARCADIA SOUTH','ROCHESTER','MOORABBIN','PUCKAPUNYAL','CRESSY','MITCHELLSTOWN','TOSTAREE','CARLYLE','JERUK','LYONVILLE','TRAFALGAR EAST','BERRIWILLOCK','MYRTLE CREEK','MULGRAVE','MOUNT BECKWORTH','GLENLOTH EAST','BOWEYA NORTH','PATYAH','ABERFELDY','LINDENOW','MOUNT PROSPECT','STOCKDALE','BARMAH','CARBOOR','NARRUNG','SMOKO','GORAE','POOWONG EAST','MAFFRA WEST UPPER','CULLULLERAINE','SWAN ISLAND','BOOLA','GUILDFORD','MANSFIELD','NOBLE PARK NORTH','MELTON WEST','MOOROOPNA','DIMBOOLA','MOORILIM','SAFETY BEACH','CRIB POINT','BUNINYONG','ELLIMINYT','HAWKESDALE','WHITELAW','APOLLO BAY','LAL LAL','LANGDONS HILL','MANORINA','MOUNT DRYDEN','MIRBOO NORTH','WOODSIDE BEACH','HILLCREST','MOYREISK','KOONDROOK','CHEWTON','MEENIYAN','KEILOR NORTH','NORTH WARRANDYTE','GRANYA','WATERWAYS','KORIELLA','TRAFALGAR SOUTH','WANDILIGONG','TARRENLEA','NULLAWIL','YATCHAW','LIMESTONE','LAKE BOGA','WOODHOUSE','YANDOIT','KALIMNA WEST','JUNGABURRA','YARRAWONGA','PLENTY','GONG GONG','HEIDELBERG HEIGHTS','MAIDEN GULLY','VICTORIA VALLEY','PIANGIL','POMBORNEIT','RAYMOND ISLAND','PINE GROVE','BENAYEO','THOWGLA VALLEY','WOOROONOOK','OMBERSLEY','SARSFIELD','WALKERVILLE SOUTH','MICKLEHAM','BET BET','MAIDSTONE','MOKEPILLY','COONOOER BRIDGE','LAKE EILDON','DEER PARK','PERRY BRIDGE','FOSTERVILLE','FULHAM','NALINGA','GLENGOWER','NEEREMAN','LERDERDERG','SEDGWICK','RAVENSWOOD','SOLDIERS HILL','WARNEET','DRIK DRIK','MINERS REST','SIDONIA','NEILBOROUGH','CHATSWORTH','ROMSEY','NORLANE','PURNIM WEST','BOLTON','THE SISTERS','COWLEYS CREEK','LAKE GARDENS','MOOLERR','BELGRAVE SOUTH','COBDEN','NOORAT','DEWHURST','SPRINGFIELD','ALLESTREE','WINJALLOK','AINTREE','KORUMBURRA SOUTH','CHUTE','BRIDGE CREEK','GOBARUP','BRIT BRIT','FAIRLEY','MUCKLEFORD','NAP NAP MARRA','TANJIL','CARDINIA','MEADOW CREEK','MOONLIGHT FLAT','SOUTHERN CROSS','WANNON','FORGE CREEK','WILSONS PROMONTORY','WATCHUPGA','BARFOLD','WANGIE','JEFFCOTT','BOOLARONG','CONGUPNA','BURNSIDE HEIGHTS','ROKEBY','BELMONT','ARTHURS SEAT','WOOLAMAI','CARRAJUNG LOWER','TALLANGATTA VALLEY','KOOROOCHEANG','GONN CROSSING','NEWINGTON','GLENBRAE','WESTMEADOWS','BYADUK','COLERAINE','SUGARLOAF','CARAPOOEE','TURRIFF EAST','NATTE YALLOCK','TARRAWARRA','MACEDON','BAGSHOT NORTH','VINIFERA','GRETA','CLARKES HILL','RYANS','TIMOR WEST','NGARKAT','WYCHITELLA NORTH','CAPE WOOLAMAI','SUTHERLANDS CREEK','ROSEBERY','FRAMLINGHAM EAST','YEO','BREAMLEA','TOOLLEEN','SANDY CREEK','WENSLEYDALE','MELVILLE FOREST','SIMPSON','ARCHERTON','BARWON HEADS','QUANTONG','SCOTTS CREEK','WANDIN NORTH','CHURCHILL','BALLARAT CENTRAL','WATTLE FLAT','PAKENHAM','KILLAWARRA','GELLIBRAND','UPPER PLENTY','CAMPBELLS BRIDGE','THOMASTOWN','KEW EAST','MEERLIEU','GYMBOWEN','MONTROSE','MOUNT MERCER','TABOR','GIRGARRE EAST','LUCYVALE','BUNGULUKE','KYVALLEY','COBRAM','MERRINEE','SWAN HILL WEST','PERONNE','DARTMOUTH','TITTYBONG','GARVOC','KEVINGTON','BREWSTER','LOCKSLEY','MALLACOOTA','WINGEEL','BIRCHIP','KALKALLO','BARRAMUNGA','RAVENHALL','LICOLA NORTH','WONGA PARK','PYALONG','NEWLYN NORTH','MEPUNGA','BELGRAVE HEIGHTS','MILAWA','GELANTIPY','TRAVANCORE','NARIEL VALLEY','OCEAN GROVE','BOOKAAR','LETHBRIDGE','BALD HILLS','DADSWELLS BRIDGE','MYERS FLAT','WANGARABELL','SPRINGVALE SOUTH','NEWMERELLA','EXFORD','PENTAL ISLAND','BARWITE','GLENGARRY WEST','FAIRHAVEN','BELLBRIDGE','BIG DESERT','COBAW','CHINKAPOOK','DOOKIE COLLEGE','WOODSTOCK ON LODDON','WONNANGATTA','MAINDAMPLE','WARRAGUL WEST','GARDENVALE','MONBULK','BLAMPIED','DUNNEWORTHY','GRUYERE','BAYLES','HOLLANDS LANDING','GIL GIL','COCKATOO','GLENAROUA','CLEAR LAKE','TYERS','BAYNTON','TOM GROGGIN','BLIND BIGHT','MONT ALBERT NORTH','SEACOMBE','WATTLE CREEK','STANHOPE','WOODGLEN','GRE GRE','JARRAHMOND','WANGOOM','JAM JERRUP','PORTARLINGTON','YARRAGON','GLEN IRIS','KOORLONG','RINGWOOD NORTH','RHYMNEY','GOLDSBOROUGH','LENEVA','MOONAMBEL','OMEO VALLEY','ERICA','MERRICKS BEACH','OAKLEIGH EAST','MOLKA','KURTING','GLENROY','BOONAH','WANDOWN','TYRENDARRA','ABERFELDIE','BAYSWATER','CAVENDISH','IVANHOE EAST','RYE','HOTHAM HEIGHTS','LANDSBOROUGH WEST','KINGSTON','ARDMONA','NUNNIONG','WHOROULY','ARMADALE','NARRE WARREN','TEMPLESTOWE','EAST WANGARATTA','MININERA','ROSEBUD','GAINSBOROUGH','MIRALIE','NORTHWOOD','FERNY CREEK','BEECH FOREST','BADDAGINNIE','TATURA','HORSHAM','WINTON','PENSHURST','COOMA','MIRIMBAH','KERRIE','NICHOLSON','RICH AVON EAST','COONGULLA','TIMBOON WEST','BALLANGEICH','BUCKRABANYULE','HOWQUA INLET','WINGAN RIVER','WATTLE HILL','HAWTHORN EAST','HUNTER','SOMERVILLE','CLARKEFIELD','RICH AVON WEST','UPOTIPOTPON','PANITYA','WYUNA','SKIPTON','PERCYDALE','TYAAK','GLENFYNE','CAULFIELD','HERNES OAK','LAKE BUNGA','BIG HILL','TINAMBA WEST','WOODEND NORTH','WESTBURY','YULECART','ARCHDALE JUNCTION','HODDLES CREEK','RED HILL','EUMEMMERRING','FORREST','LANGWARRIN','BOX HILL NORTH','HEIDELBERG WEST','KIRWANS BRIDGE','MIRBOO','EAGLEMONT','HAVEN','MAROONA','MOORABBIN AIRPORT','WUK WUK','BOLINDA','MILLGROVE','STAWELL','MURCHISON EAST','NEWPORT','CHESNEY VALE','DROUIN SOUTH','BURNBANK','SUNBURY','COMBIENBAR','DEPTFORD','BOOROOL','STRATHKELLAR','NERRINA','BIRDWOODTON','NELSE','KEELY','WERRIBEE','WHIPSTICK','SPEED','TAHARA','GERRIGERRUP','GEELONG WEST','NAMBROK','INVERLOCH','WAANYARRA','EASTERN VIEW','MYALL','GOBUR','CORAGULAC','OLINDA','WHOOREL','SWAN BAY','WALLAGARAUGH','NOORAT EAST','WHANREGARWEN','CROOKAYAN','GRANITE ROCK','GRAYS BRIDGE','WARRENBAYNE','TOOAN','NAYOOK','SIMSON','SOUTH GEELONG','MORRL MORRL','LAKE POWELL','KELLALAC','IRAAK','KINGLAKE','TIMMERING','SURREY HILLS','NARBETHONG','DROUIN EAST','ALMONDS','THE COVE','GREENWALD','KERGUNYAH SOUTH','KILMANY','GOWAR EAST','PAINSWICK','NORTH SHORE','WILLIAMS LANDING','COALVILLE','BONEGILLA','WALHALLA EAST','ANDERSON','TRESCO','ENFIELD','WANDO VALE','MOLOGA','MARLO','CHARLEROI','DOLLAR','CASTLE DONNINGTON','TRARALGON EAST','GOORNONG','IRREWILLIPE','CUNDARE NORTH','MARENGO','CORONET BAY','BARKSTEAD','MONOMEITH','SUTHERLAND','DUMBALK','LUCKNOW','NARRAWONG','DERBY','DALLAS','ILLAWARRA','DOUBLE BRIDGES','YEUNGROON EAST','MOUNT TASSIE','NILMA','DELBURN','KERRISDALE','OXLEY','WAREEK','BOLWARRAH','CLARINDA','TABBERABBERA','LAKE WONGAN','CLYDESDALE','NOTTING HILL','STRATHMERTON','BUNGEET','NUG NUG','DURDIDWARRAH','IRREWILLIPE EAST','BARRAKEE','TOURELLO','SALISBURY WEST','CARLTON NORTH','LAWLOIT','JANCOURT EAST','MINCHA','PICOLA WEST','DALYSTON','CARAPOOK','CRESWICK','MOYHU','BUNDALONG','REDCASTLE','KARABEAL','LOCKWOOD','STACEYS BRIDGE','RUNNING CREEK','STREATHAM','FISKVILLE','REEDY CREEK','PASTORIA','PORT MELBOURNE','GLADFIELD','BETE BOLONG NORTH','KUNAT','COGHILLS CREEK','REDESDALE','CROYDON HILLS','HILGAY','EAGLEHAWK NORTH','THE GURDIES','NEUARPURR','NUGGETTY','HOTSPUR','BARWON DOWNS','RHEOLA','CURDIES RIVER','CHELSEA','ALTONA','OAKLANDS JUNCTION','DEEPDENE','PEARSONDALE','HAWTHORN','SKENES CREEK NORTH','TANJIL BREN','DERRINAL','KYABRAM SOUTH','MERRIJIG','MOOLAP','MILDURA','DEVON MEADOWS','LIMA','CHILTERN','YAAPEET','LANG LANG','MEPUNGA WEST','MOUNT SABINE','BELLS BEACH','ABBOTSFORD','MANGALORE','TAYLOR BAY','WARRONG','DIGGERS REST','KENNEDYS CREEK','WELLSFORD','CORRINGLE','LORQUON','DALES CREEK','JEETHO','WEEAPROINAH','MANNS BEACH','TALLANGATTA SOUTH','CHAPPLE VALE','WOMBELANO','LAKE BOLAC','CHEWTON BUSHLANDS','REYNARD','CATHCART','DON VALLEY','ELLINBANK','YENDON','WILLIAMSTOWN NORTH','PORT FRANKLIN','STRADBROKE','BELLELLEN','BENAMBRA','BONNIE DOON','WATERFORD PARK','MILNES BRIDGE','HARCOURT NORTH','CURDIEVALE','WANGARATTA SOUTH','SEDDON','WINTER VALLEY','MOUNT EVELYN','WANDONG','LOWER MOIRA','DANDENONG SOUTH','CANNUM','SURF BEACH','GLEN WAVERLEY','WOOD WOOD','BALLAPUR','BALMATTUM','ROWVILLE','BUNGAREE','DEANS MARSH','LOCH SPORT','KATAMATITE EAST','GLENHOPE EAST','ST HELENS PLAINS','ESKDALE','YANAC','GRAMPIANS','COLDSTREAM','TAMLEUGH NORTH','JARKLIN','WALLALOO','CARALULUP','PYRAMID HILL','ESSENDON','MOUNT LONARCH','BETHANGA','FUMINA SOUTH','WARMUR','DOCKER','RIGGS CREEK','EVERTON UPPER','LUBECK','SYDENHAM','KNEBSWORTH','DUNACH','SILVERLEAVES','VENTNOR','GRAND RIDGE','HUNTERSTON','LAKE GOLDSMITH','EGANSTOWN','NUTFIELD','RIVERSIDE','MOUNT ECCLES','ASCOT VALE','KING VALLEY','BEMM RIVER','VITE VITE','TIMBARRA','ESMOND','FLYNNS CREEK','SNAKE ISLAND','MARYKNOLL','DARRAWEIT GUIM','PRESTON','PANTON HILL','BRIM','BOISDALE','BRUARONG','TARRAYOUKYAN','STAGHORN FLAT','FERNTREE GULLY','ST CLAIR','BRIDGEWATER NORTH','MOUNT TOOLEBEWONG','LILLIPUT','WOODSTOCK','LYSTERFIELD SOUTH','WOODS POINT','SHE OAKS','LOCH','PETTICOAT CREEK','NEERIM','BARKLY','DEDDICK VALLEY','HARKNESS','MINYIP','ROSEWHITE','LITTLE RIVER','COCOROC','INVERMAY','VENUS BAY','CRANBOURNE EAST','CROYDON NORTH','TERRICK TERRICK','GLENMAGGIE','ROSANNA','MOLLONGGHIP','MITCHAM','GORDON','KOROBEIT','LALOR','TONGALA','WESTMERE','MARRAWEENEY','BITTERN','CREEK JUNCTION','GIFFARD','BEAUCHAMP','WODONGA','LONG FOREST','CATHKIN','PRAIRIE','NATHALIA','NORTH BENDIGO','TARNEIT','TANYBRYN','SHEPPARTON EAST','BROOMFIELD','MELBOURNE AIRPORT','WILD DOG VALLEY','LANGWARRIN SOUTH','KALIMNA','MARYVALE','TERANG','MUSK','RIPPLESIDE','CALIFORNIA GULLY','MORANDING','BUTCHERS RIDGE','LILLIMUR','NICHOLS POINT','TYENNA','GOOMALIBEE','SAILORS FALLS','LAKE LONSDALE','BRAVINGTON','ARTHURS CREEK','NORTH BLACKWOOD','EAST BENDIGO','NOWIE','PEARCEDALE','MOSSIFACE','MODELLA','COHUNA','GREENVALE','WESBURN','TAGGERTY','DRUMANURE','ELDORADO','WILLENABRINA','LURG','WHOROULY EAST','HEALESVILLE','ROCKBANK','NIDDRIE','MITTA MITTA','TRAWOOL','MOUNT EGERTON','WAARRE','KILLARA','MIRRANATWA','CHELSEA HEIGHTS','ROWSLEY','BOWENVALE','OAKLEIGH','NAREEWILLOCK','SWANWATER','ROKEWOOD JUNCTION','YABBA SOUTH','KIMBOLTON','DIGGORA','BUNGALALLY','BENJEROOP','DORODONG','VAUGHAN','SULKY','MURRABIT','ROSE RIVER','WOODSIDE','BALLENDELLA','DUMBALK NORTH','BRANDY CREEK','KYNETON','TRAGOWEL','CLIFTON HILL','HILLSIDE','MELTON','WILSONS HILL','GHIN GHIN','MOLYULLAH','PELLUEBLA','GLENLUCE','BARUNAH PLAINS','TOORADIN','METCALFE EAST','AVONSLEIGH','BROMLEY','OAKVALE','LAWRENCE','OSBORNES FLAT','GLENLOGIE','MURROON','WANTIRNA SOUTH','WONYIP','BURAGWONDUC','WOORAGEE','WAL WAL','WHITLANDS','GLENORCHY','GOLDEN GULLY','GANNAWARRA','ANGLERS REST','PASCHENDALE','MOUNT MORIAC','WOOSANG','WILLAURA','KINNABULLA','BARINGHUP WEST','BAKERY HILL','DARRIMAN','SELWYN','WINDERMERE','BUSHFIELD','WEE WEE RUP','DOCKLANDS','DELEGATE RIVER','STRATHLEA','CLAYTON','NORVAL','MYWEE','TINAMBA','LAKE WENDOUREE','MOUNT CLEAR','MITRE','ELPHINSTONE','BEARS LAGOON','BOORHAMAN','RIPPLEBROOK','FYANS CREEK','BELLBIRD CREEK','PASTORIA EAST','TALLANDOON','RUSHWORTH','GELLIONDALE','FAIRY DELL','PARAPARAP','ST JAMES','BENTLEIGH','BORALMA','MURRUMBEENA','POOTILLA','CURLEWIS','MONEGEETTA','WINSLOW','ALTONA MEADOWS','DREEITE','BUNGAL','STRATHDOWNIE','MACORNA NORTH','NARINGAL EAST','LADYS PASS','JEERALANG','NOWHERE CREEK','CLIFTON CREEK','SPRINGHURST','ATHLONE','KARNAK','BALNARRING','KINGOWER','STRATHBOGIE','ZEERUST','EDDINGTON','NOOJEE','FRANKSTON NORTH','YINNAR','MYRTLEBANK','COOLAROO','KORWEINGUBOORA','BULLARTO','PORT FAIRY','BALINTORE','TARNOOK','BARANDUDA','GLENBURNIE','KARRAMOMUS','BROADMEADOWS','BEULAH','NHILL','VITE VITE NORTH','ANCONA','DENICULL CREEK','LAKE TYERS','JOHNSONVILLE','MOORMBOOL WEST','HATTAH','ULLSWATER','MARIBYRNONG','ASHWOOD','MITTYACK','DUTSON','WILLOW GROVE','DUNKELD','RED LION','WYUNA EAST','ARGYLE','SUNSET STRIP','COLAC','HILLDENE','KEILOR','BAW BAW','JUMBUK','CHOCOLYN','BUDGEE BUDGEE','BOOLARRA','MOGLONEMBY','WULGULMERANG WEST','MANANGATANG','SHEPPARTON','BRIDGEWATER','MOUNT DANDENONG','CLEMATIS','BENDOC','GLENROWAN','JAN JUC','LAKE CHARM','MOONEE PONDS','WOODFORD','CORACK','HUON CREEK','KANYA','MOUNT TAYLOR','WILLOWVALE','GLENLEE','WALLALOO EAST','STEELS CREEK','LONGWOOD EAST','CAMPERDOWN','POWLETT PLAINS','YALLOURN','ULLINA','TERRICK TERRICK EAST','KOTTA','BOORCAN','KILLARNEY','ROXBURGH PARK','MOOROODUC','PETERBOROUGH','HALLS GAP','NULLA VALE','ALLANSFORD','BALOOK','PANMURE','COLBROOK','REEDY FLAT','SEAHOLME','KALPIENUNG','GREENHILL','BENGWORDEN','IRONBARK','CROOKED RIVER','BRIDGEWATER ON LODDON','GREEN GULLY','BULART','MCMILLANS','HERNE HILL','ORFORD','MOE SOUTH','YOUANMITE','TYLDEN','SILVAN','WARGAN','YARPTURK','BOHO SOUTH','EAST WARBURTON','KALLISTA','YARCK','WHIRILY','LARDNER','WULGULMERANG EAST','LOCKWOOD SOUTH','FUMINA','YAMBUNA','SHERBROOKE','POMBORNEIT NORTH','ELMORE','BOOSEY','BEREMBOKE','BOLANGUM','TENBY POINT','BUNDALAGUAH','SUGARLOAF CREEK','DRIFFIELD','NUMURKAH','MADDINGLEY','PURNIM','RAYWOOD','TESBURY','MOUNTAIN BAY','LISMORE','ERRINUNDRA','BUNBARTHA','WOORINEN SOUTH','TARRA VALLEY','HEATHCOTE','STRATH CREEK','TIMBOON','BOOROLITE','TELANGATUK EAST','MUNDOONA','BRENANAH','TERRAPPEE','KOONOOMOO','ICY CREEK','MINIMAY','WATCHEM','SEVILLE','PIER MILAN','STONYFORD','KINYPANIAL','LODDON VALE','ALEXANDRA','TONGHI CREEK','BUANGOR','WESTBY','TOOBORAC','MURMUNGEE','YORK PLAINS','NOWA NOWA','ST ARNAUD','MOUNT ROWAN','DOBIE','GOUGHS BAY','CASSILIS','WEDDERBURN','BRUNSWICK','CHAPEL FLAT','POINT LEO','TRAWALLA','WARRANWOOD','CERES','MOUNT HELEN','TAMBO CROSSING','NINDA','SMOKEYTOWN','CHANDLERS CREEK','CARDIGAN','SIMPSONS CREEK','TAROON','COCAMBA','BRUMBY','WHITE HILLS','BAHGALLAH','PAKENHAM UPPER','CARNEGIE','MURRAWEE','GLEN WILLS','KOONWARRA','NULLAWARRE NORTH','SHEPPARTON NORTH','GRANGEFIELDS','ATTWOOD','GRANTVILLE','ELSTERNWICK','THORPDALE SOUTH','LEIGH CREEK','MAJOR PLAINS','AIRPORT WEST','FARADAY','RESERVOIR','NAREEB','YEODENE','KADNOOK','WERRIMULL','DENNINGTON','RUFFY','BRODRIBB RIVER','SCOTCHMANS LEAD','MALMSBURY','JUNCTION VILLAGE','CHESHUNT SOUTH','COPE COPE','WALLACE','GRITJURK','TOORA','WY YUNG','MADALYA','BALLIANG','KILSYTH SOUTH','HASTINGS','MAFFRA','WALLAN','FLAMINGO BEACH','CARDIGAN VILLAGE','UPPER FERNTREE GULLY','RHYLL','TOTTINGTON','DUNNSTOWN','PIRRON YALLOCK','MAILORS FLAT','BARRABOOL','TARWIN LOWER','SMYTHESDALE','NAVARRE','FLEMINGTON','WATCHEM WEST','PAARATTE','KATANDRA','BOWMANS FOREST','MURRA WARRA','KATUNGA','GOON NURE','WALMER','YANGERY','BOCHARA','NEWTOWN','KIALLA WEST','ESSENDON NORTH','WANGANDARY','TYRRELL','BULDAH','WUNGHNU','TUNGAMAH','BRIGHTON EAST','BARKERS CREEK','PIGGOREET','DAISY HILL','ERCILDOUNE','BUFFALO RIVER','MURRAY-SUNSET','WEST WODONGA','CAMPBELLS CREEK','MERRICKS','DARLEY','BUDGEREE','CULGOA','LARA','GREENS CREEK','LANGKOOP','PEECHELBA EAST','WARRION','KEYSBOROUGH','CARINA','BOX HILL SOUTH','BALWYN','BURKES FLAT','ULTIMA EAST','NORMANVILLE','PORTLAND WEST','KOO WEE RUP NORTH','MEATIAN','PARWAN','KIALLA','GUNDOWRING','SKENES CREEK','CROWLANDS','MARCUS HILL','BARRYS REEF','LASCELLES','WOOREEN','BARWIDGEE','SEAFORD','KORONG VALE','ASCOT','CLUNES','MOUNT BEAUTY','BALNARRING BEACH','MELTON SOUTH','NANDALY','SEYMOUR','CAPE OTWAY','POOLAIJELO','WATSONIA','ROSEBROOK','CARISBROOK','EMU','BUNG BONG','BUSHY PARK','NOORINBEE NORTH','BACCHUS MARSH','PEECHELBA','HEATHCOTE JUNCTION','MOCKINYA','MANDURANG','BUCHAN SOUTH','WHEELERS HILL','SWANPOOL','TOOLERN VALE','APSLEY','GIRGARRE','BLACKBURN NORTH','MARYSVILLE','BLOWHARD','BEVERFORD','MEAD','CRANBOURNE','MARSHALL','ORMOND','ALBERTON WEST','KOROIT','LEONGATHA','CROSS ROADS','KILMORE','ZUMSTEINS','LAKE CONDAH','ASHBOURNE','SKINNERS FLAT','NAR NAR GOON','CALDER PARK','COIMADAI','STIRLING','LEXTON','REEFTON','MOOROOPNA NORTH WEST','GLADYSDALE','CANN RIVER','LAVERTON NORTH','EAST MELBOURNE','STEWARTON','CANADIAN','SELBY','RAVENSWOOD SOUTH','WARANGA SHORES','FRYERSTOWN','MOONDARRA','CHEROKEE','HOPPERS CROSSING','NORTH WONTHAGGI','MATLOCK','FAIRBANK','YUROKE','PASCOE VALE','TARNAGULLA','KANGAROO GROUND','KONGWAK','BARRAPORT','WILLIAMSTOWN','MOUNT BUFFALO','BULLENGAROOK','BELLFIELD','KANCOONA','FERN HILL','CAULFIELD NORTH','BADGER CREEK','BEVERIDGE','VERMONT','FLAGGY CREEK','WATERHOLES','BROOKVILLE','MOLIAGUL','MURRABIT WEST','BARINGHUP','DROUIN WEST','SMITHS GULLY','TOWER HILL','TARRANYURK','TRAYNORS LAGOON','BLAKEVILLE','PENNYROYAL','GOLDEN POINT','WILKUR','BLACKBURN','BONNIE BROOK','GOWANFORD','PRANJIP','TANJIL SOUTH','DENISON','WARATAH BAY','SPRING HILL','GLENFALLOCH','QUARRY HILL','GISBORNE SOUTH','CARAPOOEE WEST','DARGO','MOUNT GLASGOW','LILLICUR','TREMONT','RICHMOND','GUYS FOREST','ALFREDTON','KOALLAH','HIGHETT','YARRAMBAT','GOORAMADDA','BUNKERS HILL','ADELAIDE LEAD','MIDDLE PARK','COOMBOONA','BRIGHT','BANDIANA','MIRAM','ARMSTRONG','POREPUNKAH','ST ALBANS','ILLABAROOK','KEILOR DOWNS','TRIDA','TARADALE','COWANGIE','BARJARG','THOLOGOLONG','AVONDALE HEIGHTS','LORNE','LONG GULLY','MARNOO','WHOROULY SOUTH','THOMSON','COCHRANES CREEK','HOPETOUN','MOUNT NAPIER','MOUNT WAVERLEY','WALLUP','TENNYSON','DELATITE','AXEDALE','THE HONEYSUCKLES','GUNYAH','GAFFNEYS CREEK','YANGOURA','BIG PATS CREEK','TECOMA','KERANG','MACLEOD','KILMORE EAST','WEST BENDIGO','BERWICK','GERMANIA','FENTONS CREEK','DONCASTER','WOODEND','NORTH WANGARATTA','HUON','HEXHAM','BOWSER','BATESFORD','SWAN REACH','FLINDERS','SPRINGVALE','MURGHEBOLUC','TYRRELL DOWNS','GLENTHOMPSON','MERNDA','LANCE CREEK','MUMBANNAR','MALVERN EAST','ST KILDA','NUNGURNER','SHELBOURNE','ULUPNA','MANOR LAKES','CRAIGIEBURN','NEWFIELD','RATHSCAR','GLENORMISTON NORTH','SORRENTO','CASTLE CREEK','GOORAMBAT','HARCOURT','CLYDEBANK','CRAIGIE','EAST GEELONG','CARLISLE RIVER','WOOLSHED FLAT','MOUTAJUP','NEDS CORNER','STONY CREEK','MOUNTAIN VIEW','TOWONG UPPER','CROYDON SOUTH','AVOCA','PIMPINIO','HEATH HILL','BENWERRIN','BLACK RANGE','BANGERANG','LINDSAY','BARNAWARTHA NORTH','DIGBY','MCCRAE','STRATHMORE HEIGHTS','CORA LYNN','LARPENT','LOWER NORTON','POWELLTOWN','STRATEN','BELLARINE','MOUNT BOLTON','RIPPONLEA','NARRE WARREN EAST','GLENROWAN WEST','WRATHUNG','TOOLONDO','RUBY','TAMBORITHA','TYNTYNDER SOUTH','MAGPIE','WARRENHEIP','LANDSBOROUGH','GUNBOWER','MCKENZIE CREEK','BERRYBANK','COWA','PENTLAND HILLS','MOOLORT','HUNTINGDALE','WOORINEN','WILLAURA NORTH','DUTSON DOWNS','STANLEY','GIPSY POINT','ALLAMBEE SOUTH','MOUNT MARTHA','NEERIM EAST','YARRAGON SOUTH','GOONGERAH','ANGLESEA','DRUMCONDRA','NULLAWARRE','NEWBOROUGH','BONBEACH','BRANXHOLME','ECHUCA WEST','BROADFORD','SOUTH PURRUMBETE','EILDON','BRINGALBERT','THORNBURY','CORNELLA','GILLIESTON','ARMSTRONG CREEK','WILBY','COLBINABBIN','FERNIHURST','MALDON','MACARTHUR','KOOLOONONG','MOUNT BULLER','MOUNT HOOGHLY','GLADSTONE PARK','REDAN','SPOTSWOOD','BALLYROGAN','ALLENDALE','LAKE ROWAN','GLENHOPE','FOREST HILL','COONOOER WEST','BANYENA','EVERTON','NURRAN','COBURG','OAK PARK','MCINTYRE','MANIFOLD HEIGHTS','VIOLET TOWN','LAUNCHING PLACE','GIFFARD WEST','TARRAVILLE','KEILOR PARK','WARBURTON','MOUNT EMU','BROKEN CREEK','HAZELWOOD NORTH','DEANSIDE','BURRAMBOOT','TALLANGATTA','WONTHAGGI','TONIMBUK','MOGGS CREEK','BURWOOD','SHADY CREEK','BUNYIP NORTH','POINT COOK','WIRRATE','BONN','BREAKAWAY CREEK','DONCASTER EAST','CANIAMBO','RUNNYMEDE','KANIVA','MANDURANG SOUTH','CORNDALE','WALWA','KNOCKWOOD','EARLSTON','KOORNALLA','CAMPBELLTOWN','MELBOURNE','MARDAN','COLAC WEST','EASTWOOD','BELGRAVE','OFFICER SOUTH','YALLAMBIE','ENSAY NORTH','ULTIMA','WYCHITELLA','MAMBOURIN','WERONA','NANNEELLA','METUNG','MARYBOROUGH','CABBAGE TREE CREEK','WANDIN EAST','SANDHILL LAKE','HANSONVILLE','BARNAWARTHA','BEENAK','NAPOLEONS','DONVALE','VALENCIA CREEK','LEONARDS HILL','LAVERS HILL','KAMAROOKA','KYABRAM','GLENORMISTON SOUTH','VICTORIA POINT','BULLAHARRE','NEWLANDS ARM','CULLA','GOOROC','PURA PURA','BEECHWORTH','WALKERVILLE','LONGLEA','HEYTESBURY LOWER','TYABB','NARINGAL','FISH POINT','AYRFORD','VECTIS','DOREEN','FRASER RISE','MUCKATAH','WINCHELSEA SOUTH','KELVIN VIEW','CUDGEWA','BLACKWARRY','CAPE SCHANCK','SMITHS BEACH','SHEEP HILLS','KARYRIE','SWAN MARSH','BORUNG','HUGHESDALE','ASPENDALE','SKYE','BUNDOORA','NAREEN','CAPE PATERSON','THORPDALE','LIMA SOUTH','DROUIN','MANNIBADAR','TANDAROOK','GLEN PARK','WOODFIELD','WOODLEIGH','WARNE','DINNER PLAIN','YEA','BULGA','YUNDOOL','KOO WEE RUP','KEILOR LODGE','TOOLANGI','QUEENSCLIFF','CAVEAT','WARROCK','GENTLE ANNIE','CHINTIN','DANDENONG NORTH','EYNESBURY','NUNAWADING','ROKEWOOD','BEAUFORT','MOUNT WALLACE','GAPSTED','DEVENISH','WON WRON','CULLEN','UPWEY','DALTONS BRIDGE','ELWOOD','GLENMORE','BAYINDEEN','FISH CREEK','DARLIMURLA','ELTHAM','CAMPBELLS FOREST','BRADVALE','BAMGANIE','BROWN HILL','MERINGUR','BULLIOH','CORUNNUN','CRESWICK NORTH','LIPAROO','DUCHEMBEGARRA','CRANBOURNE WEST','BALLIANG EAST','DERRIMUT','SHELFORD','NYARRIN','YANDO','POMPAPIEL','TANDARRA','HOWITT PLAINS','YELTA','BULN BULN','BRUCKNELL','HUNTLY','NELSON','HENSLEY PARK','KROWERA','BARONGAROOK WEST','LANGI LOGAN','CABBAGE TREE','ADDINGTON','COBBANNAH','WILLATOOK','PORTLAND','AUCHMORE','SPRINGMOUNT','ASHBURTON','RINGWOOD','GLOMAR BEACH','SMYTHES CREEK','LONDRIGAN','DAWSON','POMBORNEIT EAST','KARDELLA','STRATHTULLOH','DALYENONG','METCALFE','MAIN RIDGE','MOUNT COTTRELL','WILLUNG SOUTH','BERRINGAMA','TULKARA','WOMBAT CREEK','MIEPOLL','NYAH WEST','TULLAMARINE','MYRNIONG','DRUMBORG','PINE VIEW','SANDHURST','VESPER','HUMEVALE','KILCUNDA','KILLINGWORTH','CHIRNSIDE PARK','BAW BAW VILLAGE','PIRA','DEDERANG','BENDIGO','BURRUMBEET','LONGFORD','BYRNESIDE','SWAN HILL','LANGLEY','ARNOLD WEST','DELAHEY','TARRONE','ASPENDALE GARDENS','BRIAR HILL','BOBINAWARRAH','SEABROOK','KNOWSLEY','BELL PARK','PAYNESVILLE','GRE GRE SOUTH','POINT LONSDALE','TALGARNO','WERNETH','WISELEIGH','LAKE MUNDI','ELLASWOOD','ENDEAVOUR HILLS','BULLAROOK','DOOBOOBETIC','GRAYTOWN','WARRENMANG','SASSAFRAS','DRYSDALE','STRATHDALE','BENALLA','EDI UPPER','NERRING','MOUNT ALFRED','BIGGARA','SPRING GULLY','MURCHISON NORTH','TEMPLESTOWE LOWER','WARRAGUL SOUTH','BAMAWM','WURRUK','MARNOO WEST','WANGARATTA','MODEWARRE','WHROO','MOORNAPA','LILYDALE','EPPING','WARRAK','JACKASS FLAT','LALBERT','HIGH CAMP','MOUNT RICHMOND','HAZELWOOD','WEDDERBURN JUNCTION','GLENALBYN','BLAIRGOWRIE','HEIDELBERG','BEND OF ISLANDS','TORQUAY','WIMBLEDON HEIGHTS','SEBASTIAN','RUSSELLS BRIDGE','WATSONIA NORTH','GRASSDALE','TETOORA ROAD','CORIO','FLORA HILL','POOWONG','LANGI KAL KAL','KERNOT','HEPBURN SPRINGS','ENSAY','MOUNT FRANKLIN','TOWAN','KOTUPNA','GLENGARRY','CASTERTON','PARADISE BEACH','WOORI YALLOCK','CHERRYPOOL','BONEO','W TREE','WORROWING','CHILDERS','HARROW','CHARLTON','BALLAN','BURNSIDE','ALLAMBEE RESERVE','BALMORAL','TORWOOD','KEW','WHITEHEADS CREEK','ESSENDON WEST','NYERIMILANG','WANDO BRIDGE','JOEL JOEL','TRARALGON SOUTH','CORAL BANK','BUNGIL','CORACK EAST','CLOVER FLAT','NATYA','NEERIM NORTH','BUFFALO','FYANSFORD','COROP','GENOA','MUNRO','JEERALANG JUNCTION','CALIVIL','HAMPTON EAST','MENA PARK','ANTWERP','MOYSTON','YALLA-Y-POORA','MUNTHAM','ROBERTSONS BEACH','JUMBUNNA','PLUMPTON','MOORABOOL','EDEN PARK','WARRACKNABEAL','TYNTYNDER','KEALBA','EUSTON','TALLYGAROOPNA','LONGERENONG','PARK ORCHARDS','NINTINGBOOL','TRARALGON','MERBEIN','SNAKE VALLEY','BELL POST HILL','WATERLOO','BOXWOOD','KOOYONG','RATHSCAR WEST','WOOL WOOL','SERVICETON','FRENCHMANS','GEMBROOK','THE PATCH','BAYSWATER NORTH','CAPE CONRAN','RINGWOOD EAST','KERGUNYAH','MCMAHONS CREEK','MACORNA','HEATHMERE','GRETA SOUTH','BYADUK NORTH','KOETONG','WARTOOK','BERRIMAL','DUVERNEY','TINTALDRA','CASTLEMAINE','ELLERSLIE','FOSTER','HEATHERTON','BAROMI','FRANKSTON SOUTH','GRE GRE NORTH','TOLMIE','SEPARATION CREEK','TOORAK','TAMBO UPPER','JANCOURT','WAAIA','YALMY','DAYLESFORD','KOLORA','LESLIE MANOR','KATAMATITE','HAWKHURST','ADA','BURNLEY','EDENHOPE','CAPE BRIDGEWATER','TARRAWINGEE','TRESCO WEST','ELMHURST','BOORT','TAWONGA SOUTH','YALLOURN NORTH','GRINGEGALGONA','BONANG','LAKE FYANS','WATSONS CREEK','HEPBURN','STAFFORDSHIRE REEF','MERINO','WROXHAM','BELLBRAE','CONDAH','NIRRANDA SOUTH','COOJAR','CANARY ISLAND','AMOR','COLAC COLAC','BO PEEP','LAHARUM','BOGONG','LINDENOW SOUTH','DOOKIE','NARRAPUMELAP SOUTH','MONOMAK','LAEN EAST','KENNETT RIVER','MYRRHEE','MIOWERA','JAMIESON','DUNOLLY','THE HEART','MAIN LEAD','PORT ALBERT','BULLA','WEMEN','CORRYONG','KENLEY','LONGWOOD','WELSHPOOL','BUCHAN','CROYDON','HAMLYN HEIGHTS','ENOCHS POINT','ELEVATED PLAINS','GOLDIE','BOX HILL','KATANDRA WEST','HINNOMUNJIE','WARRAGUL','BESSIEBELLE','ST HELENA','MERRICKS NORTH','MURTOA','STRATHEWEN','LITTLE HAMPTON','BANYAN','FAWKNER','MACKS CREEK','MACCLESFIELD','KURRACA','BULLEEN','NAVIGATORS','SUNSHINE NORTH','NORTH MELBOURNE','ROSSBRIDGE','HADDON','TAMBOON','JOEL SOUTH','MOUNT DORAN','CARAG CARAG','ARDEER','LAKE MARMAL','WELSHMANS REEF','PRINCES HILL','CARRUM DOWNS','MCKENZIE HILL','NORTH GEELONG','CREEK VIEW','HIGHLANDS','WAIL','LAANECOORIE','LEONGATHA SOUTH','BUCKLEY SWAMP','CAMBERWELL','CHARLEMONT','SOUTH YARRA','WINNAMBOOL','GRETA WEST','CAMBARVILLE','AIREYS INLET','WONGA','PATCHEWOLLOCK','PATHO','TEESDALE','DRY DIGGINGS','YOUARANG','GORMANDALE','EUREKA','GERMANTOWN','MOROKA','CROXTON EAST','SPARGO CREEK','BULLARTO SOUTH','SWANWATER WEST','BRUNSWICK WEST','OMEO','SOUTH WHARF','MERRIMU','ELTHAM NORTH','AREEGRA','CASTELLA','ROSTRON','NEWBURY','NEWCOMB','SUNSHINE WEST','LINTON','WALKERVILLE NORTH','BRIMIN','CANNONS CREEK','RAGLAN','THREE BRIDGES','CARAMUT','COTTLES BRIDGE','DUNROBIN','MCLOUGHLINS BEACH','GREY RIVER','NINE MILE','EAST SALE','MONTMORENCY','ROCKY POINT','GROVEDALE','LACEBY','NAROGHID','SOMERS','SCOTSBURN','CLONBINANE','TYRENDARRA EAST','STAUGHTON VALE','MINCHA WEST','NANGANA','WAURN PONDS','MEADOW HEIGHTS','LOWER PLENTY','CARWARP','EVANSFORD','TOWANINNY SOUTH','CARLTON','TIMOR','GOORAM','WARRNAMBOOL','GARFIELD','OCEAN GRANGE','THALIA','LIMA EAST','JOHANNA','AVALON','LITTLE DESERT','MOUNT CAMERON','HARKAWAY','BURRAMINE','LEDCOURT','BOOMAHNOOMOONAH','TOOLAMBA WEST','SUNDAY CREEK','MERBEIN SOUTH','AXE CREEK','AIRE VALLEY','MIA MIA','AVENEL','NYRRABY','ELAINE','MITIAMO','SERPENTINE','CHELTENHAM','FERNBANK','EBDEN','BOOLITE','LAMPLOUGH','GLENDONALD','GUYS HILL','KIALLA EAST','WALPA','TALLAROOK','TOORONGO','NYORA','KANGAROO FLAT','CADELLO','WEST CREEK','LOGAN','EUROA','BERRYS CREEK','HOPETOUN PARK','MEPUNGA EAST','TARRENGOWER','STAVELY','PINE LODGE','REEDY DAM','YAPEEN','BAMBRA','SEASPRAY','BOORHAMAN NORTH','BRUTHEN','MOUNT DUNEED','TRENTHAM','DONALD','KINGLAKE WEST','BIRREGURRA','LOCH VALLEY','SHEPHERDS FLAT','WILLOWMAVIN','STONELEIGH','TELOPEA DOWNS','EURACK','KARDELLA SOUTH','MOUNT COLE CREEK','INVERLEIGH','SOUTH MORANG','GOWANGARDIE','AUBREY','HORFIELD','YARRA GLEN','DEVILS RIVER','NINYEUNOOK','TABILK','KIATA','WYE RIVER','SKIBO','WILLUNG','WARRABKOOK','COROROOKE','UNDERBOOL','KOROOP','NEWHAM','MAJORCA','TARRINGTON','POMONAL','WALLINGTON','YUULONG','JIL JIL','HENTY','WEST FOOTSCRAY','CUDGEE','INGLISTON','WOOTONG VALE','BROWNS PLAINS','CANTERBURY','BENA','BLACK HILL','PAKENHAM SOUTH','MORNINGTON','KAARIMBA','WYCHEPROOF','SEVILLE EAST','WEHLA','HAMILTON','WOORNDOO','LAKES ENTRANCE','ST ANDREWS','JARVIS CREEK','MERRIGUM','MUDGEGONGA','ROCKLYN','ORBOST','MORGIANA','KITHBROOK','TANGAMBALANGA','CAIRNLEA','GAZETTE','NIRRANDA EAST','MOOROOPNA NORTH','RED HILL SOUTH','MORWELL','TAWONGA','BETE BOLONG','DANDONGADALE','BONA VISTA','SHELLEY','GERANG GERUNG','CARINGAL','WEST MELBOURNE','DEAN','SEATON','EDITHVALE','CLUB TERRACE','FLOWERDALE','BOIGBEAT','BARONGAROOK','BOWEYA','WONGUNGARRA','POUND CREEK','LLOWALONG','NANGILOC','CONNEWIRRICOO','SUTTON GRANGE','COLIGNAN','ELINGAMITE','ANNUELLO','MORTON PLAINS','MONGANS BRIDGE','WOLLERT','IRREWARRA','RIVERSLEA','BENNISON','LANCASTER','GLEN ALVIE','GRASSMERE','GLEN FORBES','SUTTON','NALANGIL','SUGGAN BUGGAN','WEIR VIEWS','MINGAY','KURUNJANG','STEIGLITZ','MIDDLE CREEK','CREMORNE','INDIGO VALLEY','COKUM','BALWYN NORTH','CHINANGIN','MITCHELL PARK','MOUNT BEST','GELLIBRAND LOWER','FAIRFIELD','GLENISLA','MURRAYVILLE','PRINCETOWN','OLD TALLANGATTA','BAILIESTON','DREEITE SOUTH','RAWSON','CAULFIELD EAST','PARKVILLE','MARONG','ECHUCA VILLAGE','JUNORTOUN','ABBEYARD','VERMONT SOUTH','PORTSEA','KANAGULK','SOMERTON','BURNEWANG','KOYUGA','KENSINGTON','DRUNG','TOOTGAROOK','COSGROVE','NAGAMBIE','TOORLOO ARM','GLENLOTH','CHADSTONE','HAMPTON','NOBLE PARK','CATANI','MOUNT PLEASANT','LEMNOS','EAGLE POINT','CHIRRIP','GOWER','DOCKERS PLAINS','AVON PLAINS','NORONG','TAMINICK','DANDENONG','GOLDEN SQUARE','DIXONS CREEK','ALBANVALE','MERRIANG SOUTH','FIERY FLAT','KEILOR EAST','HEYFIELD','ROSES GAP','YINNAR SOUTH','LAH','BROOKLYN','FOXHOW','KILSYTH','HAMPTON PARK','WINNINDOO','WOOLSTHORPE','DENVER','NARRACAN','ROSLYNMEAD','NATIMUK','CAMBRIAN HILL','BIRCHIP WEST','CALLIGNEE','BALLARAT NORTH','CALLIGNEE NORTH','SLATY CREEK','ALTONA NORTH','ECKLIN SOUTH','TEDDYWADDY WEST','WAGGARANDALL','CUNDARE','WYNDHAM VALE','BURWOOD EAST','PIRIES','FERNSHAW','WURDIBOLUC','GILDEROY','OVENS','WARRAYURE','FALLS CREEK','RED CLIFFS','NANGEELA','BETLEY','CREIGHTONS CREEK','PITFIELD','YAN YEAN','BEARII','SHOREHAM','TOOLOME','VIEWBANK','BEAZLEYS BRIDGE','GLEN CREEK','GORAE WEST','MORIAC','MAFEKING','SAILORS GULLY','CAROLINE SPRINGS','LOVELY BANKS','JEPARIT','ALMA','YABBA NORTH','HIAWATHA','WALPEUP','STRATHALLAN','HOWES CREEK','HOMEBUSH','IGUANA CREEK','PRAHRAN','ARARAT','MOUNT MAJOR','MONTGOMERY','BANGHOLME','NEWSTEAD','GREAT WESTERN','GREENDALE','DINGEE','BUNDING','MURRINDAL','MARIONVALE','BURRAMINE SOUTH','PORTLAND NORTH','BULLUMWAAL','KAMAROOKA NORTH','COBAINS','STRZELECKI','KINGS PARK','HAVELOCK','KINGLAKE CENTRAL','LAKE TYRRELL','ARBUCKLE','COBBERAS','THE BASIN','ST ANDREWS BEACH','YIELIMA','WALLINDUC','ST HELENS','TYNONG','ADAMS ESTATE','CARPENDEIT','SAILORS HILL','WOOMELANG','TORRITA','BILLABONG','KEWELL','CLYDE NORTH','ROSEDALE','NEWRY','WINCHELSEA','BUCKLEY','WANDANA HEIGHTS','RIDDELLS CREEK','COSTERFIELD','BLACKWOOD','DRUMMOND NORTH','MCKINNON','WANTIRNA','BUDGERUM EAST','UPPER LURG','BRADFORD','HEYWOOD','MINJAH','DEREEL','BRIMPAEN','WARRANDYTE SOUTH','PORT CAMPBELL','BUNDARA','MARNOO EAST','COBBLEBANK','FRENCH ISLAND','ST ALBANS PARK','DERGHOLM','TONGIO','EPSOM','MARUNGI','LAWLER','BULGANA','BURROWYE','DOUGLAS','NAR NAR GOON NORTH','DOCTORS FLAT','UPTON HILL','MURPHYS CREEK','BEACONSFIELD','LOY YANG','LEAGHUR','FERGUSON','WAHGUNYAH','CHETWYND','NEERIM JUNCTION','WABONGA','JINGELLIC','YARRAWALLA','WAIREWA','SOUTH KINGSVILLE','BERRINGA','LICOLA','KENNINGTON','ST GERMAINS','YARRAVILLE','WERRIBEE SOUTH','LAKE MERAN','LYNDHURST','SAN REMO','COWES','LAURISTON']

In [15]:
# List of context words that indicate the presence of an address in the text
context_words_addr=['Address', 'Home address']
    
# Define the street recognizer pattern, matching street addresses like "123 Main St" or "45 Elm Avenue"
street_recognizer = PatternRecognizer(supported_entity="STREET",
                                              patterns = [Pattern(name="street",
                                                                  regex=r"((\w+\s(?:Alley|Ally|Arcade|Arc|Avenue|Ave|Boulevard|Bvd|Bypass|Bypa|Circuit|CCt|Close|Corner|Crn|Court|Crescent|Cres|Cul-de-sac|Cds|Drive|Esplanade|Esp|Green|Grn|Grove|Highway|Hwy|Junction|Jnc|Lane|Link|Mews|Parade|Pde|Ridge|Rdge|Road|Rd|Square|Street|St,|Place|Terrace|Tce|ALLEY|ALLY|ARCADE|ARC|AVENUE|AVE|BOULEVARD|BVD|BYPASS|BYPA|CIRCUIT|CCT|CLOSE|CORNER|CRN|COURT|CRESCENT|CRES|CUL-DE-SAC|CDS|DRIVE|ESPLANADE|ESP|GREEN|GRN|GROVE|HIGHWAY|HWY|JUNCTION|JNC|LANE|LINK|MEWS|PARADE|PDE|PLACE|RIDGE|RDGE|ROAD|RD|SQUARE|STREET|TERRACE|TCE))|(\d+\s+\w+\s(?:Alley|Ally|Arcade|Arc|Avenue|Ave|Boulevard|Bvd|Bypass|Bypa|Circuit|CCt|Close|Corner|Crn|Court|Crescent|Cres|Cul-de-sac|Cds|Drive|Esplanade|Esp|Green|Grn|Grove|Highway|Hwy|Junction|Jnc|Lane|Link|Mews|Parade|Pde|Place|Ridge|Rdge|Road|Rd|Square|Street|Terrace|Tce))|(\d+\s+\w+\s(?:Alley|Ally|Arcade|Arc|Avenue|Ave|Boulevard|Bvd|Bypass|Bypa|Circuit|CCt|Close|Corner|Crn|Court|Crescent|Cres|Cul-de-sac|Cds|Drive|Esplanade|Esp|Green|Grn|Grove|Highway|Hwy|Junction|Jnc|Lane|Link|Mews|Parade|Pde|Place|Ridge|Rdge|Road|Rd|Square|Street|Terrace|Tce|ALLEY|ALLY|ARCADE|ARC|AVENUE|AVE|BOULEVARD|BVD|BYPASS|BYPA|CIRCUIT|CCT|CLOSE|CORNER|CRN|COURT|CRESCENT|CRES|CUL-DE-SAC|CDS|DRIVE|ESPLANADE|ESP|GREEN|GRN|GROVE|HIGHWAY|HWY|JUNCTION|JNC|LANE|LINK|MEWS|PARADE|PDE|PLACE|RIDGE|RDGE|ROAD|RD|SQUARE|STREET|TERRACE|TCE))|(\D+\S+\W+\S(?:ALLEY|ALLY|ARCADE|ARC|AVENUE|AVE|BOULEVARD|BVD|BYPASS|BYPA|CIRCUIT|CCT|CLOSE|CORNER|CRN|COURT|CRESCENT|CRES|CUL-DE-SAC|CDS|DRIVE|ESPLANADE|ESP|GREEN|GRN|GROVE|HIGHWAY|HWY|JUNCTION|JNC|LANE|LINK|MEWS|PARADE|PDE|PLACE|RIDGE|RDGE|ROAD|RD|SQUARE|STREET|TERRACE|TCE)(\s+\w+\s)(?:New South Wales|Victoria|Queensland|Western Australia|South Australia|Tasmania|Australian Capital Territory|Northern Territory|NEW SOUTH WALES|VICTORIA|QUEENSLAND|WESTERN AUSTRALIA|SOUTH AUSTRALIA|TASMANIA|AUSTRALIAN CAPITAL TERRITORY|NORTHERN TERRITORY|NSW|VIC|QLD|WA|SA|TAS|ACT|NT)(\s+\d{4})))",
                                                                  score=0.5)],
                                              context=context_words_addr,)

# Define the postcode recognizer for Australian postcodes (e.g., "3000" or "3999")
postcode_recognizer_au = PatternRecognizer(supported_entity="POSTCODE_AU", 
                                        patterns = [Pattern(name="postcode", 
                                                            regex=r'\b(3[0-9]{3}|39[0-9]{2})\b',
                                                            score=0.5)], #for numbers between 4 digits long
                                        context=context_words_addr)

# Define the suburb recognizer pattern, which will not match any Victorian suburbs (as indicated by the deny list)
suburb_recognizer = PatternRecognizer(supported_entity="SUBURB",
                                      deny_list = australia_state_suburb_vic)

# Define the state recognizer, with a deny list for Australian state names and abbreviations
state_recognizer = PatternRecognizer(supported_entity="STATE",
                                     deny_list=["NSW", "New South Wales", "NEW SOUTH WALES", "QLD", "Queensland", "QUEENSLAND", "NT", "Northern Territory", "NORTHERN TERRITORY", "WA", "Western Australia", "WESTERN AUSTRALIA", "SA", "South Australia", "SOUTH AUSTRALIA", "VIC", "Victoria", "VICTORIA", "TAS","Tasmania", "TASMANIA", "ACT", "Australian Capital Territory", "AUSTRALIAN CAPITAL TERRITORY", "Australia", "AUSTRALIA"])

# Define the date recognizer, matching various date formats such as "12/12/2022" or "12th December 2022"
date_recognizer = PatternRecognizer(supported_entity="DATE",
                                patterns = [Pattern(name="date",
                                                    regex=r"([0-9]{1,2}(\/|-|.)[0-9]{1,2}(\/|-|.)[0-9]{2,4})|(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}\D?)?\D?((19[7-9]\d|20\d{2})|\d{2})",
                                                    score=0.5)])

# Define the customer address recognizer for Australian address formats (e.g., "123 Main St, VIC 3000")
cust_addr_recognizer=PatternRecognizer(supported_entity="ADDRESS_CUST",
                                patterns = [Pattern(name="date",
                                                    regex=r"(\d{1,5}\s([A-Za-z0-9\s]+)\s(Street|St|Road|Rd|Avenue|Ave|Boulevard|Blvd|Drive|Dr|Lane|Ln|Place|Pl)\s*,?\s*([A-Za-z\s]+)\s*,?\s*(NSW|VIC|QLD|WA|SA|TAS|ACT|NT)\s*,?\s*(\d{4}))", 
                                                    score=0.5)]) 

# Create a recognizer registry to store multiple recognizers
registry = RecognizerRegistry()
# Add the defined recognizers to the registry
registry.add_recognizer(date_recognizer)
registry.add_recognizer(street_recognizer)
registry.add_recognizer(postcode_recognizer_au)
registry.add_recognizer(suburb_recognizer)
registry.add_recognizer(state_recognizer)
registry.add_recognizer(cust_addr_recognizer)

# Create an analyzer engine that uses the registry to analyze text
analyzer = AnalyzerEngine(registry=registry)

In [16]:
# test recognizer
text_addr2ident="123 Elm Street. One more address: 22 Pine Lane, Western Australia 6000. Another address: 678 Beach Road, Victoria 3200 where a patirnt mover 01/01/2000 or 12th December 2022"


results = analyzer.analyze(text=text_addr2ident,
                        entities=["DATE",
                                  "STREET",
                                  "POSTCODE_AU",
                                  "SUBURB",
                                  "STATE", 
                                  "ADDRESS_CUST",
                                  ], language='en'  )  
results

[type: STATE, start: 48, end: 65, score: 1.0,
 type: STATE, start: 105, end: 113, score: 1.0,
 type: STREET, start: 0, end: 14, score: 0.5,
 type: STREET, start: 34, end: 46, score: 0.5,
 type: STREET, start: 89, end: 103, score: 0.5,
 type: POSTCODE_AU, start: 114, end: 118, score: 0.5,
 type: DATE, start: 141, end: 151, score: 0.5,
 type: DATE, start: 155, end: 173, score: 0.5]

# Phone (customized identifier)

In [17]:
# List of context words that indicate the presence of phone numbers in the text
context_words_phone=['Phone', 'Home Phone', 'Mobile Phone']

# Regex to match phone numbers with optional country code (+61, (0X), etc.), followed by 10-14 digits (for various phone formats)
phone_recognizer = PatternRecognizer(supported_entity="PHONE",
                                             patterns = [Pattern(name="phone", 
                                                                 regex=r"(\(+61\)|\+61|\(0[1-9]\)|0[1-9])?( ?-?[0-9]){10,14}",
                                                                 score=0.5)])

# Create a recognizer registry to store multiple recognizers
registry = RecognizerRegistry()
# Add the phone recognizer to the registry
registry.add_recognizer(phone_recognizer)
# Create an analyzer engine that uses the registry to analyze text
analyzer = AnalyzerEngine(registry=registry)

In [18]:
# test recognizer
text_phone2ident="Phone (Work): +61 409 123 456. Phone (Home): 0409 123 456."


results = analyzer.analyze(text=text_phone2ident,
                        entities=["PHONE"], language='en'  )  
results

[type: PHONE, start: 15, end: 29, score: 0.5,
 type: PHONE, start: 44, end: 57, score: 0.5]

# URN (customized identifier)

In [19]:
context_words_phone=['URN', 'MRN']

In [20]:
# First two numbers of the URN (Unique Reference Number) were common for all of the patients.
# The first two digits "15" are followed by 5 additional digits (which could vary for each patient).
# We will use this pattern in the recognizer to identify MRNs (Medical Record Numbers) that start with "15" followed by 5 digits.

# Define the MRN recognizer pattern
mrn_recognizer = PatternRecognizer(supported_entity="MRN",
                                             patterns = [Pattern(name="mrn", 
                                                                 regex= r"15\d{5}", 
                                                                 score=0.7)])
# Create a recognizer registry to store multiple recognizers
registry = RecognizerRegistry()
# Add the MRN recognizer to the registry
registry.add_recognizer(mrn_recognizer)
# Create an analyzer engine that uses the registry to analyze text
analyzer = AnalyzerEngine(registry=registry)

In [21]:
text_urn2ident="RN 1501235. RN: 1599999. MRN 1545670. 1545670."


results = analyzer.analyze(text=text_urn2ident,
                        entities=["MRN"], language='en'  )  
results

[type: MRN, start: 3, end: 10, score: 0.7,
 type: MRN, start: 16, end: 23, score: 0.7,
 type: MRN, start: 29, end: 36, score: 0.7,
 type: MRN, start: 38, end: 45, score: 0.7]